In [1]:
import cv2 as cv
from scanner import ROI
import scanner
import pytesseract as ocr
import parse_frame

def imshow(image):
    ww = 1024
    h, w = image.shape[:2]

    if w > ww:
        hh = int(ww / w * h)
        image = cv.resize(image, dsize=[ww, hh])

    _, ret = cv.imencode('.jpg', image) 

    from IPython.display import display, Image
    display(Image(data=ret))
cv.imshow = imshow

/Users/olir/workspace/genshin_artifact_scanner/scanner/artifacts.yaml


In [2]:
from scanner import frame_reader, ArtifactROI, ScreenState
import dataclasses
filename = frame_reader.get_test_video_path()
print(filename)
reader = frame_reader.FrameReader(filename, time=0, interval=0.02)

roi = ArtifactROI(
    main=[1214, 150, 1500, 430],
    sub=[1230, 590, 1800, 790],
    level=[1251, 530, 1324, 570],
    title=[1640, 35, 1869, 82],
)
state = ScreenState(roi)

artifacts = []

  


test_video.mp4


In [3]:
while True:
  frame = reader.next()
  if not state.is_same(frame):
    break


ArtifactROI(main=[303, 37, 375, 107], sub=[307, 147, 450, 197], level=[312, 132, 331, 142], title=[410, 8, 467, 20])


In [4]:
  # _, mask = cv.threshold(slice, 0, 255, cv.THRESH_BINARY+cv.THRESH_OTSU)
  # print(mask[0,0], mask.min())
  # # slice &= ~mask * 0xFF
  # # slice = cv.equalizeHist(slice)

  # print(repr(text)); cv.imshow(slice)

  # cv.imshow(allroi.main.clip_image(gray))
  # for roi in text_rois:
  #   cv.imshow(roi.clip_image(gray))
  # text_rois.sort(key=lambda r: r.y1)

In [5]:
from parse_frame import (
  ArtifactFields, 
  find_artifact_set_name_roi, 
  find_sub_stats_roi, 
  find_all_text_roi_in_range, 
  allroi
)

# allroi.main = ROI(1214, 230, 1600, 430)
allroi.main = ROI(1214, 230, 1550, 430)

def parse_frame(frame):
  gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
  artifact = ArtifactFields()

  # artifact set name
  set_name_roi = find_artifact_set_name_roi(frame)
  slice = set_name_roi.clip_image(gray)
  slice = 255 - slice
  text = ocr.image_to_string(slice, lang="chi_sim", config="--psm 6")
  artifact.set_name = scanner.match_artifact_set_name(text)

  # substats
  sub_stats_roi = find_sub_stats_roi(set_name_roi)
  slice = sub_stats_roi.clip_image(gray)
  text = ocr.image_to_string(slice, lang="chi_sim")
  artifact.sub_stats = scanner.match_sub_stats(text)

  # type & main stat name/value
  color_range = dict(lo=150, hi=255)
  kernel_size = 10,3
  kernel_n_iter = 5
  text_rois = find_all_text_roi_in_range(gray, allroi.main, color_range, kernel_size, kernel_n_iter)
  text_rois.sort(key=lambda r: r.y1)
  assert len(text_rois) == 3
  artifact_type, main_stat_name, main_stat_value = text_rois
  # type
  slice = artifact_type.clip_image(gray)
  text = ocr.image_to_string(slice, lang="chi_sim")

  artifact.type = scanner.match_artifact_type(text)
  # main stat name
  slice = main_stat_name.clip_image(gray)
  slice = 255 - slice
  slice = cv.equalizeHist(slice)
  text = ocr.image_to_string(slice, lang="chi_sim", config="--psm 6")
  artifact.main_stat_name = scanner.match_main_stat_name(text)
  # main stat value
  slice = main_stat_value.clip_image(gray)
  text = ocr.image_to_string(slice, config="--psm 6")
  artifact.main_stat_value = scanner.match_main_stat_value(text)

  # level
  slice = allroi.level.clip_image(gray)
  slice = 255 - slice
  text = ocr.image_to_string(slice)
  artifact.level = scanner.match_level(text)

  return artifact


info = parse_frame(frame)
info = info.__dict__
result = dict(time=reader.frame_time, info=info)
artifacts.append(result)
print(f'{reader.frame_time:.4f}', info)

0.0000 {'set_name': '流放者', 'sub_stats': {'防御力': '19', '元素精通': '13', '生命值': '3.3%', '攻击力': '12'}, 'type': '时之沙', 'main_stat_name': '攻击力', 'main_stat_value': '22.3%', 'level': '9'}
